In [1]:
from manim import *
from manim_voiceover import VoiceoverScene
from manim_voiceover.services.gtts import GTTSService

Manim Community v0.17.3

/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [192]:
%%manim -ql -v CRITICAL pascal 
#--disable_caching tt_book
#note that without --disable_caching the voice_over service will have a bug!
#Description:
#Intro scene that starts with dice rolls, and get to the question, But why is the CLT true?

COLORS = [GREY,BLUE,RED]

def LabeledTriangle(label='',my_scale=1,**triangle_args):
    tri = Triangle(**triangle_args).scale(my_scale)
    text = MathTex(label)
    text.height = tri.height/2
    text.set_x(tri.get_x())
    text.align_to(tri,DOWN)
    
    text.shift(my_scale*0.21*UP)
    tri.set_z_index(-1)
    return VGroup(tri,text)

def generate_pascals_triangle(num_rows,mod_k=None):
    #generates num_rows of pascals triangle as a numpy array.
    # if mod_k is not None, will return mod k of each row
    pascal_t = np.zeros((num_rows, num_rows), dtype=int)
    pascal_t[0,0] = 1
    for i in range(num_rows-1):
        if mod_k:
            pascal_t[i+1,:] =np.mod( pascal_t[i,:] + np.roll(pascal_t[i,:],1) , mod_k)
        else:
            pascal_t[i+1,:] = pascal_t[i,:] + np.roll(pascal_t[i,:],1)
    return pascal_t


#,fill_color = BLUE,fill_opacity=0.5
TRI = [LabeledTriangle(str(i),color=WHITE).scale(0.8) for i in range(10)]
DELAY = 0.1

class pascal(MovingCameraScene):
    def construct(self):
        n_rows = 33
        
        P_val = generate_pascals_triangle(n_rows+1)
        P = [[None for i in range(n_rows)] for j in range(n_rows)]
        P_temp = [[None for i in range(n_rows)] for j in range(n_rows)]
        P[0][0] = TRI[1].copy()
        P[0][0].to_edge(UP)
        
        w = P[0][0].width
        h = P[0][0].height
        x_init = P[0][0].get_x()
        y_init = P[0][0].get_y()
        
        
        #self.play(self.camera.frame.animate.set_height(h))
        self.play(Create(P[0][0]))
        self.wait(DELAY)
        
        
        
        #tri[0][0].color = BLUE
        
        
        
        
        for n in range(1,n_rows):
            for k in range(n+1):
                tri = LabeledTriangle(str(P_val[n,k]),color=WHITE).scale(0.8)
                P[n][k] = tri #TRI[int(P_val[n,k])].copy() #set the triangle
                P[n][k].set_x(x_init + w*(k-0.5*n)) #set the position
                P[n][k].set_y(y_init - h*n)
        
        
        def AnimateIn(n,k): 
            #animation to bring in the box [n,k]
            anim_list = []
            if k > 0:
                anim_list.append(Transform(P[n-1][k-1].copy(),P[n][k])) #left parent
            if k < n:
                anim_list.append(Transform(P[n-1][k].copy(),P[n][k])) #right parent
            return anim_list
        
        
        
        #animate in the first 5 rows, 1 row at a time
        for n in range(1,5):
            anim_list = []
            for k in range(n+1):
                anim_list.extend(AnimateIn(n,k))
            self.play(*anim_list)
            
        P_copy = P[0][0].copy()
        P_copy[0].set_fill(BLUE,opacity=1.0)
        self.play(FadeTransform(P[0][0],P_copy))
        #self.play(FadeIn(P[0][0],shift=DOWN).copy())    
        #set color for odd guys!
        for n in range(n_rows):
            for k in range(n+1):
                if P_val[n,k] % 2 == 1:
                    P[n][k][0].set_fill(BLUE, opacity=1.0)
                
        
        #reanimate first 5 rows, should add in the colors!
        for n in range(1,5):
            anim_list = []
            for k in range(n+1):
                anim_list.extend(AnimateIn(n,k))
            self.play(*anim_list)
        
        #add more rows while zooming out
        for n in range(5,n_rows):
            anim_list = []
            for k in range(n+1):
                anim_list.extend(AnimateIn(n,k))
            
            self.play(
                AnimationGroup(
                    self.camera.frame.animate.move_to(np.array([x_init,y_init-h*(n+1)/2,0])).scale(1.0+1.0/n),
                    #self.camera.frame.ScreenRectangle.animate.set_height((n+1)*h+4),
                    *anim_list,
                ),
                rate_func=rate_functions.linear
            )
            #self.play(*anim_list)
                
            #self.play(*anim_list)
        
        
        